In [ ]:
# Loading Libraries
from __future__ import print_function
import os
import random
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers
import re
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt



# setting data path data seperated by class name
training_dir = r"Pneumonia_Data/chest_xray/chest_xray/train"
testing_dir = r"Pneumonia_Data/chest_xray/chest_xray/test"
validation_dir = r"Pneumonia_Data/chest_xray/chest_xray/val"

# setting output directory
model_dir =  r"Pneumonia_Data/chest_xray/chest_xray/output/models/"
log_dir = r"Pneumonia_Data/chest_xray/chest_xray/output/logs"
# setting training parameters
norm = 255.0
rescale=1./norm
shear_range=0.2
zoom_range=0.2
horizontal_flip=True

# setting train, test, validation parameters
target_size=(224, 224)
batch_size=32
class_mode='categorical'

loss='categorical_crossentropy'
metrics=['accuracy']

epochs = 50
verbose = 1
model_file = model_dir+"weights-{epoch:02d}-val_accuracy-{val_accuracy:.2f}.hdf5"

# data generator for training
train_datagen = ImageDataGenerator(
    rescale=rescale,
    shear_range=shear_range,
    zoom_range=zoom_range,
    horizontal_flip=horizontal_flip)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode)

# data generator for validation
validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode)

# data generator for testing
test_datagen = ImageDataGenerator(rescale=rescale)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode)



# importing inception model
base_model = InceptionV3(weights='imagenet', include_top=False)

# setting model layers specially output layer with class number
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

#loading model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False



sgd = optimizers.Adam()
#print layers of inception model
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# train the top inception layers
# freeze first 249 layers
# unfreeze the rest
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# compile model with optimizer and loss
model.compile(sgd, loss=loss, metrics=metrics)
checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)
tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

tensorboard.set_model(model)
# callbacks_list = [checkpoint, tensorboard, early_stopping]
callbacks_list = [checkpoint, tensorboard]

# train inception model
# fine-tuning the top layers
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    steps_per_epoch = len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_list)

# Retraining
# setting retraining from old model
model_path = "Pneumonia_Data/chest_xray/chest_xray/output/models/"+"modelsweights-03-val_accuracy-0.89-val_loss-0.28.hdf5"
epochs=15
verbose=1

model = keras.models.load_model(model_path)

# retrain by loading last good model
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks_list)


# testing model
model_path = "Pneumonia_Data/chest_xray/chest_xray/output/models/"+"modelsweights-03-val_accuracy-0.89-val_loss-0.28.hdf5"
model = keras.models.load_model(model_path)
result = model.evaluate_generator(generator=test_generator, verbose=1)
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

# testing model
best_accuracy = 0
best_loss = 0

lowest_accuracy = 100
lowest_loss = 1000000

models_path = "Pneumonia_Data/chest_xray/chest_xray/output/models/"

model_files = os.listdir(models_path)
for model_file in model_files:
    model_path = models_path+"/"+model_file

    model = keras.models.load_model(model_path)

    result = model.evaluate_generator(generator=test_generator, verbose=1)

    accuracy = result[1]*100
    loss = result[0]

    if accuracy>best_accuracy:
        best_accuracy = accuracy
        best_loss = loss
    if loss<lowest_loss:
        lowest_accuracy = accuracy
        lowest_loss = loss
    print("-"*80)
    print(model_file + ":")
    print("-"*80)

    print("%s%.2f%s"% ("Accuracy: ", accuracy, "%"))
    print("%s%.2f"% ("Loss: ", loss))

    print("-"*80)

    print("%s%.2f%s"% ("Best Accuracy: ", best_accuracy, "%"))
    print("%s%.2f"% ("Best Loss: ", best_loss))

    print("-"*80)

    print("%s%.2f%s"% ("Lowest Accuracy: ", lowest_accuracy, "%"))
    print("%s%.2f"% ("Lowest Loss: ", lowest_loss))

    print("-"*120)
    print("-"*120)

print("_"*130)
print("_"*130)

print("%s%.2f%s"% ("Best Accuracy: ", best_accuracy, "%"))
print("%s%.2f"% ("Best Loss: ", best_loss))

print("_"*100)
print("_"*100)

print("%s%.2f%s"% ("Best Accuracy: ", lowest_accuracy, "%"))
print("%s%.2f"% ("Best Loss: ", lowest_loss))


def name_correct(name):
    return re.sub(r'[^a-zA-Z,:]', ' ', name).title()


# testing model
model_path = "Pneumonia_Data/chest_xray/chest_xray/output/models/"+"modelsweights-03-val_accuracy-0.89-val_loss-0.28.hdf5"
model = keras.models.load_model(model_path)
preds = model.predict_generator(test_generator, verbose=1, steps=len(test_generator))
y_classes = preds.argmax(axis=-1)

CM = confusion_matrix(test_generator.classes, y_classes)
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

target_names = ['Normal', 'Pneumonia']
print(classification_report(test_generator.classes, y_classes, target_names=target_names))

# Calculate Precision and Recall
tn, fp, fn, tp = CM.ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))

label_map = (test_generator.class_indices)
label_map_rev = {v: name_correct(k) for k,v in label_map.items()}
print(label_map_rev)
num_batch_t = len(validation_generator)
num = random.randint(0, num_batch_t-1)
print(num)
y_img_batch, y_class_batch = validation_generator[num]
y_pred = np.argmax(model.predict(y_img_batch),-1)
y_true = np.argmax(y_class_batch,-1)
print(sum(y_pred==y_true)/batch_size)
y_true_labels = [label_map_rev[c] for c in y_true]
y_pred_labels = [label_map_rev[c] for c in y_pred]
batch_size_t = len(y_true_labels)
fig, axs = plt.subplots(nrows=8, ncols=4, figsize=(9, 18),
                        dpi=100, facecolor='w', edgecolor='k',
                        subplot_kw={'xticks': [], 'yticks': []})

plt.rcParams.update({'axes.titlesize': 'small'})
plt.subplots_adjust(hspace=0.5, wspace=0.3)


for i in range(0, batch_size_t): # how many imgs will show from the 3x3 grid
    plt.subplot(8, 4, i+1)
    plt.imshow(y_img_batch[i])
    plt.xticks([])
    plt.yticks([])
    if y_true_labels[i]==y_pred_labels[i]:
        plt.title("org: " + y_true_labels[i] + "\npred: " + y_pred_labels[i])
    else:
        plt.title("org: " + y_true_labels[i] + "\npred: " + y_pred_labels[i], color='red')

    fig.savefig("result", dpi=100)
plt.tight_layout()
plt.show()


# randomly selected image prediction visualization
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(8, 8),
                        dpi=100, facecolor='w', edgecolor='k',
                        subplot_kw={'xticks': [], 'yticks': []})

plt.rcParams.update({'axes.titlesize': 'small'})
plt.subplots_adjust(hspace=0.5, wspace=0.3)
m = {}
for i in range(0,9): # how many imgs will show from the 3x3 grid
    num=-1
    while num in m:
        num = random.randint(0, 31)
    m[num]=1
    plt.subplot(3, 3, i+1)
    plt.imshow(y_img_batch[num])
    plt.xticks([])
    plt.yticks([])
    if y_true_labels[num]==y_pred_labels[num]:
        plt.title("org: " + y_true_labels[num] + "\npred: " + y_pred_labels[num])
    else:
        plt.title("org: " + y_true_labels[num] + "\npred: " + y_pred_labels[num], color='red')

    fig.savefig("sample", dpi=100)
plt.tight_layout()
plt.show()


